# MLPerf Inference v1.0 - Reproducing Xavier results with JetPack 4.5.1

# Table of Contents

1. [System](#system)
1. [Installation](#installation)
    1. [Clone the repo](#installation_repo)
    1. [Install the dependencies](#installation_deps)    
    1. [Link the datasets](#installation_datasets)
    1. [Download the models](#installation_models)
    1. [Build the harness](#installation_harness)
1. [ResNet50](#resnet50)
    1. [Offline](#resnet50_offline)
        1. [Build](#resnet50_offline_build)
        1. [Performance](#resnet50_offline_performance)
        1. [Accuracy](#resnet50_offline_accuracy)
    1. [SingleStream](#resnet50_singlestream)
        1. [Build](#resnet50_singlestream_build)
        1. [Performance](#resnet50_singlestream_performance)
        1. [Accuracy](#resnet50_singlestream_accuracy)
    1. [MultiStream](#resnet50_multistream)
        1. [Build](#resnet50_multistream_build)
        1. [Performance](#resnet50_multistream_performance)
        1. [Accuracy](#resnet50_multistream_accuracy)
1. [SSD-ResNet34](#ssd-resnet34)
    1. [Offline](#ssd-resnet34_offline)
        1. [Build](#ssd-resnet34_offline_build)
        1. [Performance](#ssd-resnet34_offline_performance)
        1. [Accuracy](#ssd-resnet34_offline_accuracy)
    1. [SingleStream](#ssd-resnet34_singlestream)
        1. [Build](#ssd-resnet34_singlestream_build)
        1. [Performance](#ssd-resnet34_singlestream_performance)
        1. [Accuracy](#ssd-resnet34_singlestream_accuracy)
    1. [MultiStream](#ssd-resnet34_multistream)
        1. [Build](#ssd-resnet34_multistream_build)
        1. [Performance](#ssd-resnet34_multistream_performance)
        1. [Accuracy](#ssd-resnet34_multistream_accuracy)

<a name="system"></a>
# System: [NVIDIA AGX Xavier](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/systems/AGX_Xavier_TRT.json)

## Stats: JetPack 4.5.1

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; sudo -H python3 -m pip install jetson-stats -U
...
Successfully installed jetson-stats-3.1.0
</pre>

<pre>
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; jetson_release 
  Current serial number in output stream:  19
<b> - NVIDIA Jetson AGX Xavier [16GB]</b>
   * Jetpack 4.5.1 [L4T 32.5.1]
   * NV Power Mode: <font color="#859900">MAXN</font> - Type: <font color="#859900">0</font>
   * jetson_stats.service: <font color="#859900">active</font>
<b> - Libraries:</b>
   * CUDA: 10.2.89
   * cuDNN: 8.0.0.180
   * TensorRT: 7.1.3.0
   * Visionworks: 1.6.0.501
   * OpenCV: 4.1.1 compiled CUDA: <font color="#DC322F">NO</font>
   * VPI: ii libnvvpi1 1.0.15 arm64 NVIDIA Vision Programming Interface library
   * Vulkan: 1.2.70
</pre>

## OS: Ubuntu 18.04.5 LTS

<pre>
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; cat /etc/lsb-release
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=18.04
DISTRIB_CODENAME=bionic
DISTRIB_DESCRIPTION=&quot;Ubuntu 18.04.5 LTS&quot;
</pre>

## CPU: 8-core ARMv8 @ 2265 MHz

<pre>
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; lscpu
Architecture:        aarch64
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  1
Core(s) per socket:  2
Socket(s):           4
Vendor ID:           Nvidia
Model:               0
Model name:          ARMv8 Processor rev 0 (v8l)
Stepping:            0x0
CPU max MHz:         2265.6001
CPU min MHz:         115.2000
BogoMIPS:            62.50
L1d cache:           64K
L1i cache:           128K
L2 cache:            2048K
L3 cache:            4096K
Flags:               fp asimd evtstrm aes pmull sha1 sha2 crc32 atomics fphp asimdhp
</pre>

## GPU

<pre>
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; sudo jetson_clocks
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; sudo jetson_clocks --show
SOC family:tegra194  Machine:Jetson-AGX
Online CPUs: 0-7
cpu0: Online=1 Governor=schedutil MinFreq=2265600 MaxFreq=2265600 CurrentFreq=2265600 IdleStates: C1=0 c6=0 
cpu1: Online=1 Governor=schedutil MinFreq=2265600 MaxFreq=2265600 CurrentFreq=2265600 IdleStates: C1=0 c6=0 
cpu2: Online=1 Governor=schedutil MinFreq=2265600 MaxFreq=2265600 CurrentFreq=2265600 IdleStates: C1=0 c6=0 
cpu3: Online=1 Governor=schedutil MinFreq=2265600 MaxFreq=2265600 CurrentFreq=2265600 IdleStates: C1=0 c6=0 
cpu4: Online=1 Governor=schedutil MinFreq=2265600 MaxFreq=2265600 CurrentFreq=2265600 IdleStates: C1=0 c6=0 
cpu5: Online=1 Governor=schedutil MinFreq=2265600 MaxFreq=2265600 CurrentFreq=2265600 IdleStates: C1=0 c6=0 
cpu6: Online=1 Governor=schedutil MinFreq=2265600 MaxFreq=2265600 CurrentFreq=2265600 IdleStates: C1=0 c6=0 
cpu7: Online=1 Governor=schedutil MinFreq=2265600 MaxFreq=2265600 CurrentFreq=2265600 IdleStates: C1=0 c6=0 
GPU MinFreq=1377000000 MaxFreq=1377000000 CurrentFreq=1377000000
EMC MinFreq=204000000 MaxFreq=2133000000 CurrentFreq=2133000000 FreqOverride=1
Fan: PWM=0
NV Power Mode: MAXN
</pre>


<pre>
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; ck compile program:tool-print-cuda-devices
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; ck run program:tool-print-cuda-devices
...
GPU Device ID: 0
GPU Name: Xavier
GPU compute capability: 7.2
CUDA driver version: 10.2
CUDA runtime version: 10.2
Global memory: 33479647232
Max clock rate: 1377.000000 MHz
Total amount of shared memory per block: 49152
Total number of registers available per block: 65536
Warp size: 32
Maximum number of threads per multiprocessor:  2048
Maximum number of threads per block: 1024
Max dimension size of a thread block X: 1024
Max dimension size of a thread block Y: 1024
Max dimension size of a thread block Z: 64
Max dimension size of a grid size X: 2147483647
Max dimension size of a grid size Y: 65535
Max dimension size of a grid size Z: 65535
</pre>


## Disks

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; df -h
Filesystem      Size  Used Avail Use% Mounted on
/dev/mmcblk0p1   28G   17G  9.4G  64% /
none             16G     0   16G   0% /dev
tmpfs            16G   56K   16G   1% /dev/shm
tmpfs            16G   42M   16G   1% /run
tmpfs           5.0M  4.0K  5.0M   1% /run/lock
tmpfs            16G     0   16G   0% /sys/fs/cgroup
/dev/mmcblk1p1  361G  314G   28G  92% /sd
tmpfs           3.2G   12K  3.2G   1% /run/user/120
tmpfs           3.2G     0  3.2G   0% /run/user/1000
</pre>

<a name="installation"></a>
# Installation

<a name="installation_repo"></a>
## Clone the v1.0 results repo

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; git clone https://github.com/mlcommons/inference_results_v1.0.git</pre>

<a name="installation_deps"></a>
## Install the dependencies

To [quote](https://github.com/mlcommons/inference_results_v1.0/tree/master/closed/NVIDIA#prerequisites) NVIDIA:
> Note that this might take a while, on the order of several hours.

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; chmod +x inference_results_v1.0/closed/NVIDIA/scripts/install_xavier_dependencies.sh
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; time sudo inference_results_v1.0/closed/NVIDIA/scripts/install_xavier_dependencies.sh
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; python3 -m pip install onnx
</pre>

<a name="installation_datasets"></a>
## Link the datasets

We reused the datasets we generated while reproducing the v0.5 results.

<pre>
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; export MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; mkdir &dollar;MLPERF_SCRATCH_PATH
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; cd &dollar;MLPERF_SCRATCH_PATH
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/mlperf_scratch_path</b></font>&dollar; ln -s /datasets/inference_results_v0.5-nvidia/closed/NVIDIA/build/preprocessed_data preprocessed_data
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/mlperf_scratch_path</b></font>&dollar; ln -s /datasets/inference_results_v0.5-nvidia/closed/NVIDIA/build/data data
<font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/mlperf_scratch_path</b></font>&dollar; ls -la /datasets/mlperf_scratch_path/
total 20
drwxrwsr-x  3 anton dvdt 4096 Jan 28 11:10 <font color="#268BD2"><b>.</b></font>
drwxrwsr-x 17 root  dvdt 4096 Jan 28 09:53 <font color="#268BD2"><b>..</b></font>
lrwxrwxrwx  1 anton dvdt   64 Jan 28 11:10 <font color="#2AA198"><b>data</b></font> -&gt; <font color="#268BD2"><b>/datasets/inference_results_v0.5-nvidia/closed/NVIDIA/build/data</b></font>
drwxrwsr-x  5 anton dvdt 4096 Jan 28 09:54 <font color="#268BD2"><b>models</b></font>
lrwxrwxrwx  1 anton dvdt   77 Jan 28 11:10 <font color="#2AA198"><b>preprocessed_data</b></font> -&gt; <font color="#268BD2"><b>/datasets/inference_results_v0.5-nvidia/closed/NVIDIA/build/preprocessed_data</b></font>
</pre>

<a name="installation_models"></a>
## Download the models

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; git diff Makefile
<b>diff --git a/closed/NVIDIA/Makefile b/closed/NVIDIA/Makefile</b>
<b>index e7647663..2b9356f6 100644</b>
<b>--- a/closed/NVIDIA/Makefile</b>
<b>+++ b/closed/NVIDIA/Makefile</b>
<font color="#2AA198">@@ -360,7 +360,7 @@</font> endif
 
 ############################## DOWNLOAD_MODEL ##############################
 
<font color="#DC322F">-BENCHMARKS = resnet50 ssd-resnet34 ssd-mobilenet bert dlrm rnnt 3d-unet</font>
<font color="#859900">+BENCHMARKS = resnet50 ssd-resnet34 ssd-mobilenet # bert dlrm rnnt 3d-unet</font>
 
 .PHONY: download_model
 download_model: link_dirs

<font color="#2AA198">@@ -474,7 +474,6 @@</font> ifeq (&dollar;(ARCH), x86_64)
        cd build/plugins/DLRMBottomMLPPlugin \
                &amp;&amp; cmake -DCMAKE_BUILD_TYPE=&dollar;(BUILD_TYPE) &dollar;(PROJECT_ROOT)/code/plugin/DLRMBottomMLPPlugin \
                &amp;&amp; make -j
<font color="#DC322F">-endif</font>
        mkdir -p build/plugins/RNNTOptPlugin
        cd build/plugins/RNNTOptPlugin \
                &amp;&amp; cmake -DCMAKE_BUILD_TYPE=&dollar;(BUILD_TYPE) &dollar;(PROJECT_ROOT)/code/plugin/RNNTOptPlugin \
<font color="#2AA198">@@ -491,6 +490,7 @@</font> endif
        cd build/plugins/conv3D1X1X1K4Plugin \
                &amp;&amp; cmake -DCMAKE_BUILD_TYPE=&dollar;(BUILD_TYPE) &dollar;(PROJECT_ROOT)/code/plugin/conv3D1X1X1K4Plugin \
                &amp;&amp; make -j
<font color="#859900">+endif</font>
 
 # Build LoadGen.
 .PHONY: build_loadgen
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path make download_model
...
Finished downloading all the models!
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; make build
...
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>$ ls -la build/
total 28
drwxrwsr-x  7 anton dvdt 4096 Apr 27 11:48 <font color="#268BD2"><b>.</b></font>
drwxrwsr-x 14 anton dvdt 4096 Apr 27 11:29 <font color="#268BD2"><b>..</b></font>
drwxrwsr-x  2 anton dvdt 4096 Apr 27 11:49 <font color="#268BD2"><b>bin</b></font>
lrwxrwxrwx  1 anton dvdt   34 Apr 27 11:48 <font color="#2AA198"><b>data</b></font> -&gt; <font color="#268BD2"><b>/datasets/mlperf_scratch_path/data</b></font>
drwxrwsr-x  5 anton dvdt 4096 Apr 27 11:49 <font color="#268BD2"><b>harness</b></font>
drwxrwsr-x 16 anton dvdt 4096 Apr 27 11:24 <font color="#268BD2"><b>inference</b></font>
lrwxrwxrwx  1 anton dvdt   36 Apr 27 11:48 <font color="#2AA198"><b>models</b></font> -&gt; <font color="#268BD2"><b>/datasets/mlperf_scratch_path/models</b></font>
drwxrwsr-x  5 anton dvdt 4096 Apr 27 11:27 <font color="#268BD2"><b>plugins</b></font>
drwxrwsr-x  7 anton dvdt 4096 Apr 27 11:24 <font color="#268BD2"><b>power-dev</b></font>
lrwxrwxrwx  1 anton dvdt   47 Apr 27 11:48 <font color="#2AA198"><b>preprocessed_data</b></font> -&gt; <font color="#268BD2"><b>/datasets/mlperf_scratch_path/preprocessed_data</b></font>
</pre>

<a name="resnet50"></a>
# ResNet50

<a name="resnet50_offline"></a>
## Offline

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/resnet50/Offline/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/resnet50/Offline/config.json -A 15
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;concurrent_offline_expected_qps&quot;: 2181,
        &quot;config_ver&quot;: {
            &quot;maxq&quot;: {
                &quot;concurrent_offline_expected_qps&quot;: 1530
            }
        },
        &quot;dla_batch_size&quot;: 32,
        &quot;dla_copy_streams&quot;: 1,
        &quot;dla_core&quot;: 0,
        &quot;dla_inference_streams&quot;: 1,
        &quot;dla_offline_expected_qps&quot;: 396,
        &quot;gpu_batch_size&quot;: 64,
        &quot;gpu_copy_streams&quot;: 1,
        &quot;gpu_offline_expected_qps&quot;: 1478.33
    },
</pre>

<a name="resnet50_offline_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=offline&quot;
...
[2021-04-27 17:07:37,429 main.py:112 INFO] Finished building engines for resnet50 benchmark in Offline scenario.
Time taken to generate engines: 65.42682266235352 seconds

real    1m6.766s
user    0m26.980s
sys     0m6.420s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name resnet50-Offline*.plan -exec du -hs {} \;
48M     ./build/engines/AGX_Xavier/resnet50/Offline/resnet50-Offline-gpu-b64-int8.default.plan
33M     ./build/engines/AGX_Xavier/resnet50/Offline/resnet50-Offline-dla-b32-int8.default.plan
</pre>

<a name="resnet50_offline_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/resnet50/Offline/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per second: 2071.51
Result is : VALID
</pre>

#### Reproduced experiment (~53 W)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=offline --test_mode=PerformanceOnly&quot;
...
================================================
MLPerf Results Summary
================================================
SUT name : LWIS_Server
Scenario : Offline                
Mode     : PerformanceOnly      
Samples per second: 2086.92     
Result is : VALID        
  Min duration satisfied : Yes
  Min queries satisfied : Yes 
                            
================================================
Additional Stats         
================================================
Min latency (ns)                : 155755993
Max latency (ns)                : 689754096859
Mean latency (ns)               : 345039571231           
50.00 percentile latency (ns)   : 345089223369                                                        
90.00 percentile latency (ns)   : 620810076791
95.00 percentile latency (ns)   : 655270079960                       
97.00 percentile latency (ns)   : 669065261726
99.00 percentile latency (ns)   : 682850845028
99.90 percentile latency (ns)   : 689068993553                   

================================================
Test Parameters Used                                             
================================================
samples_per_query : 1439460    
target_qps : 2181                                         
target_latency (ns): 0
max_async_queries : 1
min_duration (ms): 600000
max_duration (ms): 0
min_query_count : 1
max_query_count : 0                                        
qsl_rng_seed : 7322528924094909334                         
sample_index_rng_seed : 1570999273408051088                
schedule_rng_seed : 3507442325620259414                    
accuracy_log_rng_seed : 0
accuracy_log_probability : 0
accuracy_log_sampling_target : 0
print_timestamps : 0
performance_issue_unique : 0
performance_issue_same : 0
performance_issue_same_index : 0
performance_sample_count : 2048

No warnings encountered during test.

No errors encountered during test.

Finished running actual test.
Device Device:0 processed:
  1 batches of size 36
  15545 batches of size 64
  Memcpy Calls: 0
  PerSampleCudaMemcpy Calls: 10368
  BatchedCudaMemcpy Calls: 15384
Device Device:0.DLA-0 processed:
  6938 batches of size 32
  Memcpy Calls: 0
  PerSampleCudaMemcpy Calls: 0
  BatchedCudaMemcpy Calls: 0
Device Device:0.DLA-1 processed:
  6954 batches of size 32
  Memcpy Calls: 0
  PerSampleCudaMemcpy Calls: 0
  BatchedCudaMemcpy Calls: 0
&&&& PASSED Default_Harness # ./build/bin/harness_default
[2021-04-27 17:22:17,724 main.py:280 INFO] Result: result_samples_per_second: 2086.92, Result is VALID

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-Offline:
    resnet50: result_samples_per_second: 2086.92, Result is VALID


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-Offline:
    resnet50: No accuracy results in PerformanceOnly mode.


real    11m52.559s
user    11m30.528s
sys     0m12.236s
</pre>

<a name="resnet50_offline_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/resnet50/Offline/accuracy/accuracy.txt)

<pre>
accuracy=76.074%, good=38037, total=50000
hash=3dc4add63a23f3f1f23e44abbf6b5e8f9c8a12e44412e4aa6a77f09cac22eee2
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=offline --test_mode=AccuracyOnly&quot;
...
accuracy=76.040%, good=38020, total=50000

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-Offline:
    resnet50: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-Offline:
    resnet50: Accuracy = 76.040, Threshold = 75.695. Accuracy test PASSED.


real    3m29.147s
user    0m58.836s
sys     0m17.444s
</pre>

<a name="resnet50_singlestream"></a>
## SingleStream

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/resnet50/SingleStream/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/resnet50/SingleStream/config.json -A 6
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;config_ver&quot;: {
            &quot;maxq&quot;: {}
        },
        &quot;gpu_single_stream_expected_latency_ns&quot;: 2273000,
        &quot;use_graphs&quot;: false
    },
</pre>

<a name="resnet50_singlestream_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=singlestream&quot;
...
[2021-04-27 17:35:34,949 main.py:112 INFO] Finished building engines for resnet50 benchmark in SingleStream scenario.
Time taken to generate engines: 42.95994567871094 seconds

real    0m44.209s
user    0m23.192s
sys     0m4.620s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name resnet50-SingleStream*.plan -exec du -hs {} \;
44M     ./build/engines/AGX_Xavier/resnet50/SingleStream/resnet50-SingleStream-gpu-b1-int8.default.plan
</pre>

<a name="resnet50_singlestream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/resnet50/SingleStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
90th percentile latency (ns) : 1980582
Result is : VALID
  Min duration satisfied : Yes
  Min queries satisfied : Yes
...
QPS w/ loadgen overhead         : 509.00
QPS w/o loadgen overhead        : 512.43
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=SingleStream --test_mode=PerformanceOnly&quot;
...
================================================
MLPerf Results Summary            
================================================
SUT name : LWIS_Server    
Scenario : Single Stream 
Mode     : Performance
90th percentile latency (ns) : 2078719
Result is : VALID               
  Min duration satisfied : Yes          
  Min queries satisfied : Yes              

================================================    
Additional Stats
================================================
QPS w/ loadgen overhead         : 485.07
QPS w/o loadgen overhead        : 488.48

Min latency (ns)                : 1980890       
Max latency (ns)                : 15165301
Mean latency (ns)               : 2047146
50.00 percentile latency (ns)   : 2038621
90.00 percentile latency (ns)   : 2078719
95.00 percentile latency (ns)   : 2098912
97.00 percentile latency (ns)   : 2118465
99.00 percentile latency (ns)   : 2187844
99.90 percentile latency (ns)   : 2564821

================================================
Test Parameters Used
================================================
samples_per_query : 1
target_qps : 439.947
target_latency (ns): 0
max_async_queries : 1
min_duration (ms): 60000
max_duration (ms): 0
min_query_count : 1024
max_query_count : 0
qsl_rng_seed : 12786827339337101903
sample_index_rng_seed : 12640797754436136668
schedule_rng_seed : 3135815929913719677
accuracy_log_rng_seed : 0
accuracy_log_probability : 0
accuracy_log_sampling_target : 0
print_timestamps : false
performance_issue_unique : false
performance_issue_same : false
performance_issue_same_index : 0
performance_sample_count : 2048

No warnings encountered during test.

No errors encountered during test.
Finished running actual test.
Device Device:0 processed:
  29106 batches of size 1
  Memcpy Calls: 0
  PerSampleCudaMemcpy Calls: 0
  BatchedCudaMemcpy Calls: 29106
&&&& PASSED Default_Harness # ./build/bin/harness_default
[2021-01-29 23:25:15,883 main.py:341 INFO] Result: 90th percentile latency (ns) : 2078719 and Result is : VALID

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-SingleStream:
    resnet50: 90th percentile latency (ns) : 2078719 and Result is : VALID


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-SingleStream:
    resnet50: No accuracy results in PerformanceOnly mode.


real    1m16.374s
user    1m6.484s
sys     0m5.004s

</pre>

<a name="resnet50_singlestream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/resnet50/SingleStream/accuracy/accuracy.txt)

<pre>
accuracy=76.064%, good=38032, total=50000
hash=7458cd3f1154670a0d063c87b38d2eba7aa8c1921f2558a46333cfef8d9b4036
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=SingleStream --test_mode=AccuracyOnly&quot;
...
accuracy=76.078%, good=38039, total=50000

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-SingleStream:
    resnet50: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-SingleStream:
    resnet50: Accuracy = 76.078, Threshold = 75.695. Accuracy test PASSED.


real    4m27.773s
user    2m9.392s
sys     0m11.332s
</pre>

<a name="resnet50_multistream"></a>
## MultiStream

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/resnet50/MultiStream/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/resnet50/MultiStream/config.json -A 14
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;concurrent_multi_stream_samples_per_query&quot;: 96,
        &quot;config_ver&quot;: {
            &quot;maxq&quot;: {}
        },
        &quot;dla_batch_size&quot;: 15,
        &quot;dla_copy_streams&quot;: 2,
        &quot;dla_core&quot;: 0,
        &quot;dla_inference_streams&quot;: 4,
        &quot;dla_multi_stream_samples_per_query&quot;: 15,
        &quot;gpu_batch_size&quot;: 66,
        &quot;gpu_copy_streams&quot;: 2,
        &quot;gpu_inference_streams&quot;: 4,
        &quot;gpu_multi_stream_samples_per_query&quot;: 66
    },
</pre>

<a name="resnet50_multistream_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=multistream&quot;
...
[2021-04-27 20:34:39,925 main.py:112 INFO] Finished building engines for resnet50 benchmark in MultiStream scenario.
Time taken to generate engines: 118.26688885688782 seconds

real    1m59.516s
user    0m35.180s
sys     0m15.528s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name resnet50-MultiStream*.plan -exec du -hs {} \;
96M     ./build/engines/AGX_Xavier/resnet50/MultiStream/resnet50-MultiStream-gpu-b66-int8.default.plan
29M     ./build/engines/AGX_Xavier/resnet50/MultiStream/resnet50-MultiStream-dla-b15-int8.default.plan
</pre>

<a name="resnet50_multistream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/resnet50/MultiStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per query : 96
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=MultiStream --test_mode=PerformanceOnly&quot;
[2021-04-27 20:38:49,061 main.py:701 INFO] Detected System ID: AGX_Xavier                             
[2021-04-27 20:38:49,062 main.py:529 INFO] Using config files: configs/resnet50/MultiStream/config.json
[2021-04-27 20:38:49,063 __init__.py:341 INFO] Parsing config file configs/resnet50/MultiStream/config.json ...
[2021-04-27 20:38:49,063 main.py:542 INFO] Processing config "AGX_Xavier_resnet50_MultiStream"
[2021-04-27 20:38:49,064 main.py:224 INFO] Running harness for resnet50 benchmark in MultiStream scenario...
...
</pre>

<a name="resnet50_multistream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/resnet50/MultiStream/accuracy/accuracy.txt)

<pre>
accuracy=76.008%, good=38004, total=50000
hash=12988c6c56d7af58b5e816d9b9d64b20f5ae80497e355dc51653ed2433b583bb
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=MultiStream --test_mode=AccuracyOnly&quot;
...
accuracy=76.020%, good=38010, total=50000

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-MultiStream:
    resnet50: requested_multi_stream_samples_per_query: 96, Result validity unknown


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-MultiStream:
    resnet50: Accuracy = 76.020, Threshold = 75.695. Accuracy test PASSED.


real    1m2.343s
user    0m24.100s
sys     0m7.468s
</pre>

<a name="ssd-resnet34"></a>
# SSD-ResNet34

<a name="ssd-resnet34_offline"></a>
## Offline

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/ssd-resnet34/Offline/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/ssd-resnet34/Offline/config.json -A 18
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;concurrent_offline_expected_qps&quot;: 50,
        &quot;dla_batch_size&quot;: 1,
        &quot;dla_copy_streams&quot;: 1,
        &quot;dla_core&quot;: 0,
        &quot;dla_inference_streams&quot;: 1,
        &quot;dla_offline_expected_qps&quot;: 10,
        &quot;gpu_batch_size&quot;: 2,
        &quot;gpu_copy_streams&quot;: 4,
        &quot;gpu_inference_streams&quot;: 1,
        &quot;gpu_offline_expected_qps&quot;: 35.1243,
        &quot;input_dtype&quot;: &quot;int8&quot;,
        &quot;input_format&quot;: &quot;linear&quot;,
        &quot;map_path&quot;: &quot;data_maps/coco/val_map.txt&quot;,
        &quot;min_query_count&quot;: 24576,
        &quot;precision&quot;: &quot;int8&quot;,
        &quot;tensor_path&quot;: &quot;${PREPROCESSED_DATA_DIR}/coco/val2017/SSDResNet34/int8_linear&quot;,
        &quot;use_graphs&quot;: false
    },
</pre>

<a name="ssd-resnet34_offline_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=offline&quot;
...
[2021-01-29 17:52:19,860 main.py:153 INFO] Finished building engines for ssd-resnet34 benchmark in Offline scenario.
Time taken to generate engines: 464.4344849586487 seconds

real    7m46.577s
user    2m12.112s
sys     0m15.832s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-resnet34-Offline*.plan -exec du -hs {} \;
148M    ./build/engines/AGX_Xavier/ssd-resnet34/Offline/ssd-resnet34-Offline-gpu-b2-int8.default.plan
23M     ./build/engines/AGX_Xavier/ssd-resnet34/Offline/ssd-resnet34-Offline-dla-b1-int8.default.plan
</pre>

<a name="ssd-resnet34_offline_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/Offline/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per second: 50.9757
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=Offline --test_mode=PerformanceOnly&quot;
...
================================================ 
MLPerf Results Summary
================================================
SUT name : LWIS_Server
Scenario : Offline 
Mode : Performance
Samples per second: 50.8099
Result is : VALID
Min duration satisfied : Yes
Min queries satisfied : Yes

================================================
Additional Stats 
================================================
Min latency (ns): 58655035
Max latency (ns): 483684969003 
Mean latency (ns) : 241873131399
50.00 percentile latency (ns) : 241862755973
90.00 percentile latency (ns) : 435313276430 
95.00 percentile latency (ns) : 459447810260
97.00 percentile latency (ns) : 469144908922
99.00 percentile latency (ns) : 478833559170 
99.90 percentile latency (ns) : 483161611392

================================================ 
Test Parameters Used
================================================
samples_per_query : 24576 
target_qps : 50
target_latency (ns): 0
max_async_queries : 1
min_duration (ms): 60000
max_duration (ms): 0
min_query_count : 1 
max_query_count : 0
qsl_rng_seed : 12786827339337101903
sample_index_rng_seed : 12640797754436136668
schedule_rng_seed : 3135815929913719677
accuracy_log_rng_seed : 0
accuracy_log_probability : 0
accuracy_log_sampling_target : 0
print_timestamps : false
performance_issue_unique : false
performance_issue_same : false
performance_issue_same_index : 0
performance_sample_count : 64

No warnings encountered during test.

No errors encountered during test.
Finished running actual test.
Device Device:0 processed:
8193 batches of size 2
Memcpy Calls: 0
PerSampleCudaMemcpy Calls: 0
BatchedCudaMemcpy Calls: 8193
Device Device:0.DLA-0 processed:
4095 batches of size 1
Memcpy Calls: 0
PerSampleCudaMemcpy Calls: 0
BatchedCudaMemcpy Calls: 0
Device Device:0.DLA-1 processed:
4095 batches of size 1
Memcpy Calls: 0
PerSampleCudaMemcpy Calls: 0
BatchedCudaMemcpy Calls: 0
&&&& PASSED Default_Harness # ./build/bin/harness_default
[2021-01-29 22:55:27,391 main.py:341 INFO] Result: Samples per second: 50.8099 and Result is : VALID

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-Offline:
ssd-resnet34: Samples per second: 50.8099 and Result is : VALID


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-Offline:
ssd-resnet34: No accuracy results in PerformanceOnly mode.


real8m15.281s
user8m8.460s
sys 0m6.888s
</pre>

<a name="ssd-resnet34_offline_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/Offline/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.380
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.331
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.352
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.428
mAP=20.048%
hash=95b218c56f20ce3e77d5af209df4b8219bf86c6339aebf93ac3d954126f60fc9
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=Offline --test_mode=AccuracyOnly&quot;
...
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.380
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.331
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.352
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.428
mAP=20.057%

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-Offline:
    ssd-resnet34: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-Offline:
    ssd-resnet34: Accuracy = 20.057, Threshold = 19.800. Accuracy test PASSED.


real    8m52.357s
user    5m58.536s
sys     0m15.148s
</pre>

<a name="ssd-resnet34_singlestream"></a>
## SingleStream

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/ssd-resnet34/SingleStream/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/ssd-resnet34/SingleStream/config.json -A 11
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;gpu_batch_size&quot;: 1,
        &quot;gpu_copy_streams&quot;: 1,
        &quot;gpu_inference_streams&quot;: 1,
        &quot;gpu_single_stream_expected_latency_ns&quot;: 29478000,
        &quot;input_dtype&quot;: &quot;int8&quot;,
        &quot;input_format&quot;: &quot;linear&quot;,
        &quot;map_path&quot;: &quot;data_maps/coco/val_map.txt&quot;,
        &quot;precision&quot;: &quot;int8&quot;,
        &quot;tensor_path&quot;: &quot;${PREPROCESSED_DATA_DIR}/coco/val2017/SSDResNet34/int8_linear&quot;,
        &quot;use_graphs&quot;: false
    },
</pre>

<a name="ssd-resnet34_singlestream_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=singlestream&quot;
...
[2021-01-29 17:45:05,185 main.py:153 INFO] Finished building engines for ssd-resnet34 benchmark in SingleStream scenario.
Time taken to generate engines: 63.35827445983887 seconds

real    1m5.118s
user    0m15.400s
sys     0m5.076s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-resnet34-SingleStream*.plan -exec du -hs {} \;
37M     ./build/engines/AGX_Xavier/ssd-resnet34/SingleStream/ssd-resnet34-SingleStream-gpu-b1-int8.default.plan
</pre>

<a name="ssd-resnet34_singlestream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/SingleStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
90th percentile latency (ns) : 28531845
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=SingleStream --test_mode=PerformanceOnly&quot;
...
================================================
MLPerf Results Summary
================================================
SUT name : LWIS_Server    
Scenario : Single Stream
Mode     : Performance
90th percentile latency (ns) : 28554901
Result is : VALID  
  Min duration satisfied : Yes   
  Min queries satisfied : Yes     

================================================
Additional Stats
================================================
QPS w/ loadgen overhead   : 35.17 
QPS w/o loadgen overhead  : 35.23

Min latency (ns)    : 28109905
Max latency (ns)    : 32556991
Mean latency (ns)   : 28385289
50.00 percentile latency (ns)   : 28361924
90.00 percentile latency (ns)   : 28554901
95.00 percentile latency (ns)   : 28625392
97.00 percentile latency (ns)   : 28688816
99.00 percentile latency (ns)   : 28884610
99.90 percentile latency (ns)   : 29821107

================================================
Test Parameters Used
================================================
samples_per_query : 1
target_qps : 33.9236
target_latency (ns): 0
max_async_queries : 1
min_duration (ms): 60000
max_duration (ms): 0
min_query_count : 1024
max_query_count : 0
qsl_rng_seed : 12786827339337101903
sample_index_rng_seed : 12640797754436136668
schedule_rng_seed : 3135815929913719677
accuracy_log_rng_seed : 0
accuracy_log_probability : 0
accuracy_log_sampling_target : 0
print_timestamps : false
performance_issue_unique : false
performance_issue_same : false
performance_issue_same_index : 0
performance_sample_count : 64

No warnings encountered during test.

No errors encountered during test.
Finished running actual test.
Device Device:0 processed:
  2111 batches of size 1
  Memcpy Calls: 0
  PerSampleCudaMemcpy Calls: 0
  BatchedCudaMemcpy Calls: 2111
&&&& PASSED Default_Harness # ./build/bin/harness_default
[2021-01-29 22:39:22,362 main.py:341 INFO] Result: 90th percentile latency (ns) : 28554901 and Result is : VALID

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-SingleStream:
    ssd-resnet34: 90th percentile latency (ns) : 28554901 and Result is : VALID


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-SingleStream:
    ssd-resnet34: No accuracy results in PerformanceOnly mode.


real    1m9.437s
user    1m4.720s
sys     0m3.964s
</pre>

<a name="ssd-resnet34_singlestream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/SingleStream/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.430
mAP=20.111%
hash=526aac286ebb67218a3528397b4aecbff9269cbe01307069569345d9c3fbb445
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=SingleStream --test_mode=AccuracyOnly&quot;
...
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.430
mAP=20.111%

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-SingleStream:
    ssd-resnet34: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-SingleStream:
    ssd-resnet34: Accuracy = 20.111, Threshold = 19.800. Accuracy test PASSED.


real    12m55.238s
user    8m38.988s
sys     0m16.408s
</pre>

<a name="ssd-resnet34_multistream"></a>
## MultiStream

### [Config](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/configs/ssd-resnet34/MultiStream/config.json)

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; grep AGX_Xavier /datasets/inference_results_v1.0/closed/NVIDIA/configs/ssd-resnet34/MultiStream/config.json -A 9
    &quot;<font color="#DC322F"><b>AGX_Xavier</b></font>&quot;: {
        &quot;gpu_batch_size&quot;: 2,
        &quot;gpu_multi_stream_samples_per_query&quot;: 2,
        &quot;input_dtype&quot;: &quot;int8&quot;,
        &quot;input_format&quot;: &quot;linear&quot;,
        &quot;map_path&quot;: &quot;data_maps/coco/val_map.txt&quot;,
        &quot;precision&quot;: &quot;int8&quot;,
        &quot;tensor_path&quot;: &quot;${PREPROCESSED_DATA_DIR}/coco/val2017/SSDResNet34/int8_linear&quot;,
        &quot;use_graphs&quot;: false
    },
</pre>

<a name="ssd-resnet34_multistream_build"></a>
### Build

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=multistream&quot;
...
[2021-01-29 07:33:48,592 main.py:153 INFO] Finished building engines for ssd-resnet34 benchmark in MultiStream scenario.
Time taken to generate engines: 263.8264467716217 seconds

real    4m25.496s
user    0m32.000s
sys     0m35.784s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-resnet34-MultiStream*.plan -exec du -hs {} \;
148M    ./build/engines/AGX_Xavier/ssd-resnet34/MultiStream/ssd-resnet34-MultiStream-gpu-b2-int8.default.plan
</pre>

<a name="ssd-resnet34_multistream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/MultiStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per query : 2
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=MultiStream --test_mode=PerformanceOnly&quot;
...
================================================
MLPerf Results Summary
================================================
SUT name : LWIS_Server     
Scenario : Multi Stream                
Mode     : Performance   
Samples per query : 2       
Result is : VALID               
  Performance constraints satisfied : Yes
  Min duration satisfied : Yes  
  Min queries satisfied : Yes 
               
================================================
Additional Stats
================================================
Intervals between each IssueQuery:  "qps" : 15, "ms" : 66.6667
50.00 percentile : 1              
90.00 percentile : 1         
95.00 percentile : 1      
97.00 percentile : 1      
99.00 percentile : 1      
99.90 percentile : 1
             
Per-query latency:  "target_ns" : 66666666, "target_ms" : 66.6667
50.00 percentile latency (ns)   : 55624812               
90.00 percentile latency (ns)   : 55913099 
95.00 percentile latency (ns)   : 56006924
97.00 percentile latency (ns)   : 56067621          
99.00 percentile latency (ns)   : 56189283
99.90 percentile latency (ns)   : 56401684
          
Per-sample latency:
Min latency (ns)                : 55180175
Max latency (ns)                : 61310689      
Mean latency (ns)               : 55646311
50.00 percentile latency (ns)   : 55624812
90.00 percentile latency (ns)   : 55913099   
95.00 percentile latency (ns)   : 56006924
97.00 percentile latency (ns)   : 56067621
99.00 percentile latency (ns)   : 56189283
99.90 percentile latency (ns)   : 56401684

================================================
Test Parameters Used
================================================
samples_per_query : 2
target_qps : 15
target_latency (ns): 66666666
max_async_queries : 1
min_duration (ms): 60000
max_duration (ms): 0
min_query_count : 270336
max_query_count : 0
qsl_rng_seed : 12786827339337101903
sample_index_rng_seed : 12640797754436136668
schedule_rng_seed : 3135815929913719677
accuracy_log_rng_seed : 0
accuracy_log_probability : 0
accuracy_log_sampling_target : 0
print_timestamps : false
performance_issue_unique : false
performance_issue_same : false
performance_issue_same_index : 0
performance_sample_count : 64

No warnings encountered during test.

No errors encountered during test.
Finished running actual test.
Equivalent QPS computed by samples_per_query*target_qps : 30
Device Device:0 processed:
  270336 batches of size 2
  Memcpy Calls: 0
  PerSampleCudaMemcpy Calls: 0
  BatchedCudaMemcpy Calls: 270336
&&&& PASSED Default_Harness # ./build/bin/harness_default
[2021-01-29 13:12:12,503 main.py:341 INFO] Result: Samples per query : 2 and Result is : VALID

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-MultiStream:
    ssd-resnet34: Samples per query : 2 and Result is : VALID


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-MultiStream:
    ssd-resnet34: No accuracy results in PerformanceOnly mode.


real    300m37.676s
user    5m11.760s
sys     0m27.856s
</pre>

<a name="ssd-resnet34_multistream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v1.0/blob/master/closed/NVIDIA/results/AGX_Xavier_TRT/ssd-resnet34/MultiStream/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.430
mAP=20.111%
hash=859242388c9a94513b189eb58a55bd11ad1d2f7d094880dfb72157a7ac5e45fd
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v1.0/closed/NVIDIA</b></font>&dollar; time \
MLPERF_SCRATCH_PATH=/datasets/mlperf_scratch_path \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=MultiStream --test_mode=AccuracyOnly&quot;
...
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.430
mAP=20.111%

======================= Perf harness results: =======================

AGX_Xavier_TRT-default-MultiStream:
    ssd-resnet34: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

AGX_Xavier_TRT-default-MultiStream:
    ssd-resnet34: Accuracy = 20.111, Threshold = 19.800. Accuracy test PASSED.


real    11m44.285s
user    4m22.432s
sys     0m17.144s
</pre>